In [1]:
from pyspark.sql import types as T
from pyspark.sql import functions as F 
from pyspark.sql import DataFrame
from pyspark.sql.functions import * 
from pyspark.sql.window import Window
import sys

# from pyspark.sql.types import IntegerType
# from pyspark.sql.functions import first

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
54,application_1588792046371_0175,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Table 1A+1B+2 
registrations_raw = spark.sql("""
    select 
        UID.id as user_id
        , to_date(dcreated) as reg_date
        , passwordset
        , subscriptionid
        , to_date(subscriptiondate) as sub_date
        , isactive
        , manualunsubscribe
        , to_date(logdatetime) as log_date
        , logaction
        , count(logdatetime) as n_events
    from stepstonede.userlogin UID
    left join stepstonede.subscription SUB
    on UID.id = SUB.userlogin_id
    left join stepstonede.logmystst LOG
    on 
        UID.id = LOG.userloginid 
        and logaction in ("MyStSt,AccountValidation","AUM,AccountValidation", "AUM,confirmation", "IJM, subscription", "SIM_LastOpen", "IJM, autodeactivate", "IJM, autoactivate", 
            "IJM, Manual Deactivate", "IJM, activation", "IJM, unsubscription", "JA,activation") --- added "JA,activation" and "AUM,confirmation" 
    where
        dcreated >= '2020-01-01' 
    group by
        UID.id 
        , reg_date
        , passwordset
        , subscriptionid
        , sub_date
        , isactive
        , manualunsubscribe
        , log_date
        , logaction
    order by reg_date, log_date""")


#"JA,activation" added to logaction list

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# New: these are the ratios i spoke about - count of distinct user id values
registrations_raw.agg(F.countDistinct("user_id").alias("registrations"),
                      F.countDistinct(F.when(F.col("logaction").isin(["MyStSt,AccountValidation","AUM,AccountValidation","JA, activation","AUM,confirmation"]), F.col("user_id"))).alias("accountactivation"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, subscription"]), F.col("user_id"))).alias("IJMSubscription"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, unsubscription"]), F.col("user_id"))).alias("IJMunsubscribe"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, autoactivate"]), F.col("user_id"))).alias("IJMautoactivation"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, autodeactivate"]), F.col("user_id"))).alias("IJMautodeactivation"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, Manual Deactivate"]), F.col("user_id"))).alias("IJMManualDeactivation"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, activation"]), F.col("user_id"))).alias("IJMactivation")).show()
# from this we get ratios to validate:
# - Account activations = 253000/400225 = 63% this seems to be OK
# - IJMSubscriptions = 232952 / 400225 = this seems about right + check against subscription table
# - IJMunsubscribe = 22 this number is much too low! This log doesnt seem to be written correctly
# - IJMautoactivation = roughly 1/4 of registrations: seems very low
# - IJMautodeactivation - similar magnitude as IJMautoactivation - does seem to make sense
# - IJMactivation - does not seem to be used anymore

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# here with changed logic for IJM subscription and IJMunsubscribe
registrations_raw.agg(F.countDistinct("user_id").alias("registrations"),
                      F.countDistinct(F.when(F.col("logaction").isin(["MyStSt,AccountValidation","AUM,AccountValidation","JA, activation","AUM,confirmation"]), F.col("user_id"))).alias("accountactivation"),
                      F.countDistinct(F.when(F.col("subscriptionid").isNotNull(), F.col("user_id"))).alias("IJMSubscription"),
                      F.countDistinct(F.when(F.col("manualunsubscribe") == True, F.col("user_id"))).alias("IJMunsubscribe"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, autoactivate"]), F.col("user_id"))).alias("IJMautoactivation"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, autodeactivate"]), F.col("user_id"))).alias("IJMautodeactivation"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, Manual Deactivate"]), F.col("user_id"))).alias("IJMManualDeactivation"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, activation"]), F.col("user_id"))).alias("IJMactivation")).show()
# ijm subscription yields similar number as above - so both logmystst and subscription table seem to be ok to use
# ijm unsubscription captured in subscription table - but not sure if we can find a timestamp for this? we know from another analysis that majority of unsubscribes are done on day 1 of receiving an email, 
# so we could use subscription date as the unsubscribe date

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----------------+---------------+--------------+-----------------+-------------------+---------------------+-------------+
|registrations|accountactivation|IJMSubscription|IJMunsubscribe|IJMautoactivation|IJMautodeactivation|IJMManualDeactivation|IJMactivation|
+-------------+-----------------+---------------+--------------+-----------------+-------------------+---------------------+-------------+
|       411564|           256165|         244069|         42723|            96508|              96458|                    0|            0|
+-------------+-----------------+---------------+--------------+-----------------+-------------------+---------------------+-------------+

In [5]:
# Conclusions:
# we assume account validation is stored correctly in logactions ["MyStSt,AccountValidation","AUM,AccountValidation","JA, activation","AUM,confirmation"]
# we take the IJM subscription information from logactions
# we take the IJM unsubscribe action from the subscription table and assume the unsubscribe happens on the day of the subscription
# we include ijmautoactivation and ijmdeactivation in the analysis

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#UPDATED!!!
# Table 2: validated accounts
registration_df = registrations_raw.select("user_id", "reg_date").distinct().\
    join(registrations_raw.select("user_id", "logaction", "log_date"). \
        filter(F.col("logaction").isin(["MyStSt,AccountValidation","AUM,AccountValidation","JA, activation","AUM,confirmation"])). \
        groupBy("user_id"). \
        agg(F.min("log_date").alias("AccountValidation")), "user_id", "left")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#CHECK!
registration_df.agg(F.countDistinct("user_id").alias("regs"),
                    F.countDistinct(F.when(F.col("AccountValidation").isNotNull(), F.col("user_id"))).alias("validated_accounts")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------------------+
|  regs|validated_accounts|
+------+------------------+
|430867|            260605|
+------+------------------+

In [4]:
# UPDATED!!!!
# Table2A: Validated accounts & considered_dates
# create DF with all dates from first day to last day per user
# user_df =
considered_date_df = registrations_raw.select("user_id", "reg_date"). \
    distinct(). \
    withColumn("first_day", F.to_date(F.col("reg_date"))). \
    withColumn("last_day", F.to_date(F.lit(registrations_raw.select(F.max(F.col("reg_date")).alias("max_date")).collect()[0][0]))). \
    withColumn("diffDays",  F.datediff("last_day", "first_day")). \
    withColumn("repeat", F.expr("split(repeat(',', diffDays), ',')")).\
    select("*", F.posexplode("repeat").alias("daycount", "val")).\
    drop("repeat", "val", "diffDays").\
    withColumn("considered_date", F.expr("date_add(first_day, daycount)")).\
    select("user_id", "considered_date")

validated_considered_df = registration_df.join(considered_date_df, "user_id", "left").\
    withColumn("validated_account", F.when(F.col("AccountValidation")<=F.col("considered_date"), F.lit(1)).otherwise(F.lit(0))).\
    select("user_id", "reg_date", "considered_date", "validated_account")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# CHECK!
validated_considered_df.groupBy("considered_date").\
    agg(F.countDistinct("user_id").alias("regs"),
        F.countDistinct(F.when(F.col("validated_account") == 1, F.col("user_id"))).alias("validated_accounts")).\
    sort(F.col("considered_date")).show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----+------------------+
|considered_date| regs|validated_accounts|
+---------------+-----+------------------+
|     2020-01-01| 1450|               893|
|     2020-01-02| 4698|              3039|
|     2020-01-03| 7956|              5358|
|     2020-01-04|10539|              7263|
|     2020-01-05|13565|              9407|
|     2020-01-06|17678|             12291|
|     2020-01-07|21818|             15318|
|     2020-01-08|25981|             18343|
|     2020-01-09|29514|             20956|
|     2020-01-10|32363|             23132|
|     2020-01-11|34606|             24827|
|     2020-01-12|37807|             27217|
|     2020-01-13|41949|             30303|
|     2020-01-14|46146|             33423|
|     2020-01-15|49998|             36263|
|     2020-01-16|53799|             39117|
|     2020-01-17|56675|             41229|
|     2020-01-18|58990|             42837|
|     2020-01-19|61872|             44891|
|     2020-01-20|66023|             47907|
+----------

In [5]:
#UPDATED!!!
# Table2B: IJM subscription 
IJM_subscription_df = registrations_raw. \
    filter(F.col("logaction").isin(["IJM, subscription"])). \
    select("user_id", "log_date", "n_events").\
    withColumn("n_events", F.when(F.col("n_events").isNull(), F.lit(0)).otherwise(F.col("n_events")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#UPDATED!!!
#Table3.1 (considered_subscribed_df): Join Table2a(validated_considered_df) and Table2B(IJM_subscription_df) 
#on User ID and considered_date = subscription date
#join type: left (please confirm)

#register intermidiate tables to be able to use sql
validated_considered_df.registerTempTable("validated_considered_df")
IJM_subscription_df.registerTempTable("IJM_subscription_df")

#join Table2A (validated_considered_df) to Table2B (IJM_subscription_df) 
#on 'user_id' and considered_date = subscription date

considered_subscribed_df1 = spark.sql("""
    select *
    from validated_considered_df as df1
    left join IJM_subscription_df as df2
    on df1.user_id = df2.user_id
    and df1.considered_date = df2.log_date
    """
)

considered_subscribed_df = considered_subscribed_df1.select(validated_considered_df.user_id,
                                                            validated_considered_df.reg_date,
                                                            'validated_account', 
                                                            'considered_date', 
                                                            col('n_events').alias('sub_events')).distinct()

considered_subscribed_df = considered_subscribed_df.fillna({'sub_events':'0'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
# CHECK!
considered_subscribed_df.groupBy("considered_date").\
    agg(F.countDistinct("user_id").alias("regs"),
        F.countDistinct(F.when(F.col("validated_account") == 1, F.col("user_id"))).alias("validated_accounts"),
        F.countDistinct(F.when(F.col("sub_events") >= 1, F.col("user_id"))).alias("new_subscriptions")).\
    sort(F.col("considered_date")).show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----+------------------+-----------------+
|considered_date| regs|validated_accounts|new_subscriptions|
+---------------+-----+------------------+-----------------+
|     2020-01-01| 1450|               893|              674|
|     2020-01-02| 4698|              3039|             1734|
|     2020-01-03| 7956|              5358|             1907|
|     2020-01-04|10539|              7263|             1508|
|     2020-01-05|13565|              9407|             1699|
|     2020-01-06|17678|             12291|             2346|
|     2020-01-07|21817|             15317|             2537|
|     2020-01-08|25980|             18342|             2581|
|     2020-01-09|29512|             20955|             2303|
|     2020-01-10|32361|             23131|             1873|
|     2020-01-11|34604|             24826|             1441|
|     2020-01-12|37805|             27216|             2021|
|     2020-01-13|41947|             30302|             2756|
|     2020-01-14|46144| 

In [7]:
# UPDATED!!!
#Table2C: (IJM_unsubscription_df)
IJM_unsubscription_df = registrations_raw. \
    filter(F.col("manualunsubscribe") == True). \
    select("user_id", "manualunsubscribe").distinct()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# UPDATED!!! we use anti_join, since we dont have an unsubscribe timestamp, that means all users who ever unscrubscribed are removed from the data!
#Table3.2 (considered_unsubscribed_df): Join Table3.1(considered_subscribed_df) and Table2C(IJM_unsubscription_df) 
#on User ID 
#subscribe the tables to be able to use sql to join them
considered_subscribed_df.registerTempTable("considered_subscribed_df")
IJM_unsubscription_df.registerTempTable("IJM_unsubscription_df")

considered_unsubscribed_df = considered_subscribed_df.join(IJM_unsubscription_df, "user_id", "left_anti").\
    withColumn("sub_events", F.when(F.col("sub_events")>=1, F.lit(1)).otherwise(F.col("sub_events"))).\
    select("user_id", "reg_date", "validated_account", "considered_date", "sub_events")

considered_unsubscribed_df = considered_unsubscribed_df.fillna({'sub_events':'0'})


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# UPDATED!!!!
#Table2D: IJM_autoactivate_df
IJM_autoactivate_df = registrations_raw. \
    filter(F.col("logaction").isin(["IJM, autoactivate"])). \
    select("user_id", "log_date", "n_events").distinct()

IJM_autoactivate_df = IJM_autoactivate_df.fillna({'n_events':'0'})

#IJM_autoactivate_df.show(n=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# UPDATED!!!
# Table3.3 (considered_autoactivated_df): Join Table3.2(considered_unsubscribed_df) and Table2D(IJM_autoactivate_df) 
# on User ID and considered_date = autoactivation date
    
#subscribe the tables to be able to use sql to join them
considered_unsubscribed_df.registerTempTable("considered_unsubscribed_df")
IJM_autoactivate_df.registerTempTable("IJM_autoactivate_df")

considered_autoactivated_df = spark.sql("""
    select df1.user_id, reg_date, validated_account, considered_date, sub_events, n_events as autoactivated_events
    from considered_unsubscribed_df as df1
    left join IJM_autoactivate_df as df2
    on df1.user_id = df2.user_id
    and df1.considered_date = df2.log_date""")

considered_autoactivated_df = considered_autoactivated_df.fillna({'autoactivated_events':'0'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# UPDATED!!!
#Table2E: IJM_autodeactivate_df
IJM_autodeactivate_df = registrations_raw. \
    filter(F.col("logaction").isin(["IJM, autodeactivate"])). \
    select("user_id","log_date", "n_events")

IJM_autodeactivate_df = IJM_autodeactivate_df.fillna({'n_events':'0'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# UPDATED!!!
# Table3.4 (considered_autodeactivated_df): Join Table3.3(considered_autoactivated_df) and Table2E(IJM_deautoactivate_df)
# on User ID and considered_date = autodeactivate date
# join type: left (please confirm)  
    
#subscribe the tables to be able to use sql to join them
considered_autoactivated_df.registerTempTable("considered_autoactivated_df")
IJM_autodeactivate_df.registerTempTable("IJM_autodeactivate_df")

email_log_df = spark.sql("""
    select df1.user_id, reg_date, validated_account, considered_date, sub_events, autoactivated_events, n_events as autodeactivate_events
    from considered_autoactivated_df as df1
    left join IJM_autodeactivate_df as df2
    on df1.user_id = df2.user_id
    and df1.considered_date = df2.log_date
"""
)

email_log_df = email_log_df.fillna({'autodeactivate_events':'0'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# UPDATED!!!
#email_log_df: Add the cumulative sum of 'sub_sum'
email_log_df = email_log_df.select("user_id", "reg_date", "validated_account", "considered_date", "sub_events", "autoactivated_events", "autodeactivate_events"). \
    distinct(). \
    withColumn('sub_sum', F.sum(email_log_df.sub_events).over(Window.partitionBy('user_id'). \
        orderBy('considered_date').rowsBetween(-sys.maxsize, 0))) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# UPDATED!!!
#email_log_df: Add the cumulative sum of 'autoactivated_sum'
email_log_df = email_log_df.select("user_id", "reg_date", "validated_account", "considered_date", "sub_events", "autoactivated_events", "autodeactivate_events", "sub_sum"). \
    distinct(). \
    withColumn('autoactivated_sum', F.sum(email_log_df.autoactivated_events).over(Window.partitionBy('user_id'). \
        orderBy('considered_date').rowsBetween(-sys.maxsize, 0)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# UPDATED!!!
#email_log_df: Add the cumulative sum of 'autodeactivate_sum'
email_log_df = email_log_df.select("user_id", "reg_date", "validated_account", "considered_date", "sub_events", "autoactivated_events", "autodeactivate_events", "sub_sum", "autoactivated_sum"). \
    distinct(). \
    withColumn('autodeactivate_sum', F.sum(email_log_df.autodeactivate_events).over(Window.partitionBy('user_id'). \
        orderBy('considered_date').rowsBetween(-sys.maxsize, 0)))

#email_log_df.show(n=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# UPDATED!!!
email_log_df = email_log_df.withColumn("sub_state", F.when(F.col("sub_sum")>=1, F.lit(1)).otherwise(F.lit(0))).\
    withColumn('autoactivate_state', F.when((col("autoactivated_sum")-col("autodeactivate_sum")) <= 0,F.lit(0)).otherwise(F.lit(1)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:

email_log_df = email_log_df.select("user_id", "reg_date", "validated_account", "considered_date", "sub_sum", "autoactivated_sum", "autodeactivate_sum", "sub_state"). \
    distinct(). \
    withColumn('FourtyFiveDaysAgo', F.date_sub('considered_date', 45))

#email_log_df.show(n=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
#Output: Get rid of autoactivate_state and autoactivated_sum and instead add the UK_rule_considered(countDistinct of user_id if UK_rule_considered ==1 )

email_log_df.groupBy("considered_date").\
    agg(F.countDistinct("user_id").alias("registrations"),
        F.countDistinct(F.when(F.col("validated_account") >=1, F.col("user_id"))).alias("validated_accounts"),
        F.countDistinct(F.when(F.col("sub_state") >=1, F.col("user_id"))).alias("subscriptions"),
        F.countDistinct(F.when(F.col("autoactivated_sum") >=1, F.col("user_id"))).alias("activated_subscriptions_v0"),
        F.countDistinct(F.when(F.col("autoactivate_state") >=1, F.col("user_id"))).alias("activated_subscriptions")).\
    sort("considered_date").show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------------+------------------+-------------+--------------------------+-----------------------+
|considered_date|registrations|validated_accounts|subscriptions|activated_subscriptions_v0|activated_subscriptions|
+---------------+-------------+------------------+-------------+--------------------------+-----------------------+
|     2020-01-01|         1260|               738|          519|                         0|                      0|
|     2020-01-02|         4067|              2486|         1833|                         0|                      0|
|     2020-01-03|         6823|              4361|         3262|                         0|                      0|
|     2020-01-04|         9046|              5938|         4409|                         1|                      1|
|     2020-01-05|        11652|              7686|         5669|                         1|                      1|
|     2020-01-06|        15148|             10005|         7382|        

In [18]:
#Table3: Website_visits_df

Website_visits_df = spark.sql("""
    select distinct user_id, user_hash, job_agent_id, to_date(visit_start_time_gmt) as visit_date
    from presentation.hits_de
    where exclude_hit <= 0

    union all

    select distinct user_id, user_hash, job_agent_id, to_date(visit_start_time_gmt) as visit_date
    from presentation.hits_android_global AA
    where AA.job_site like '%-de' and exclude_hit <= 0

    union all

    select distinct user_id, user_hash, job_agent_id, to_date(visit_start_time_gmt) as visit_date
    from presentation.hits_iphoneapp_global AA
    where AA.job_site like '%-de' and exclude_hit <= 0 """)

#Website_visits_df.show(n=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
"""
Here some more details on the website visit data and why there is a high number of rows with a missing user_id.

We essentially have 3 user identifiers in the website visit data:
-	User_id: this is only available if the user is logged in – and often is not the case
-	Jobagent_id: this is set, when a user lands on the website from the jobagent emails. Every jobagent_id can be linked to exactly 
one user_id. There is a table called stepstonede.userlogin_jobuser that stores this relationship. 
Here the userlogin_id is our user_id and the jobuser_id is our jobagent_id (https://eureka.stepstone.com/display/DBA/Userlogin_jobuser)
-	Userhash_id: this is a user id derived from the cookie in a device. This id cannot always be linke to exactly one user_id 
(it is essentially a m:n relationship). There is a table called presentation.adobe_user_id_hash_de which contains the relationships. 
You could query this table and look for user_hash_ids which are linked to exactly one user_id.

So – to match more website visits we could prioritise these IDs as follows:
1)	If there is a user_id in the data, use it.
2)	If there is a jobagent_id, match that jobagent_id to a user_id
3)	If there is only a user_hash_id then match it to a user_id only if that user_hash_id is used with only one user_id

If this doesn’t help to match enough users, then we might also want to look at tables presentation.ssa_fact 
and presentation.ssa_user_id_mapping_dim. But I’d suggest we try to above first, and then look at these tables afterwards.

"""
#data information:
#number of rows in Website_visits_df: 609,045,102
#number of not null user_id in Website_visits_df: 468,626,019
# Hence, we have 140,419,083 rows in Website_visits_df that the user_id is null
#number of rows in jobuser_df: 13,034,505 (distinct)
#number of rows in Userhash_df: 20,154,975
# Hence, the best case scenario we can expect to fill 33,189,480 rows out of 140,419,083 null rows of Website_visits_df 

#Test the join result:
#number of Not null rows in Website_visit_final_df would be around (468,626,019 - 33,189,480 = 435,436,539) 
#if all of the rows in jobuser_df and Userhash_df join successfully with Website_visits_df 

In [19]:
#To deal with null user_id in Website_visits_df we use Jobagent_id to match user_id from jobuser_df
#Jobagent_id: match jobagent_id from stepstonede.userlogin_jobuser to user_id in Website_visits_df 

jobuser_original_df = spark.sql("""
    select distinct userlogin_id, jobuser_id as job_agent_id
    from stepstonede.userlogin_jobuser
    """)
#So – to match more website visits we could prioritise these IDs as follows:
#1)	If there is a user_id in the data, use it.
#2)	If there is a jobagent_id, match that jobagent_id to a user_id

# this can be simplified:
jobuser_df = jobuser_original_df.join(Website_visits_df.filter(F.col("user_id").isNull()).select("job_agent_id").distinct(), "job_agent_id").distinct()

#Website_visits_df.registerTempTable("Website_visits_df")
#jobuser_original_df.registerTempTable("jobuser_original_df")

#jobuser_df = spark.sql("""
#    WITH job_agent as (SELECT job_agent_id, userlogin_id FROM jobuser_original_df Group By job_agent_id, userlogin_id)
#    SELECT distinct Website_visits_df.job_agent_id, job_agent.userlogin_id 
#    FROM Website_visits_df
#    INNER JOIN job_agent ON Website_visits_df.job_agent_id = job_agent.job_agent_id and Website_visits_df.user_id is Null
#""")

#jobuser_df.show(n=5)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
jobuser_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[job_agent_id: int, userlogin_id: int]

In [20]:
#Website_visits_jobuser_df: right join jobuser_df and Website_visits_df on job_agent_id to fill in the null user_id with user_id_updated
# this can also be written as
Website_visits_jobuser_df = Website_visits_df.join(jobuser_df, "job_agent_id", "left")


#jobuser_df.registerTempTable("jobuser_df")

#Website_visits_jobuser_df = spark.sql("""
#    select *
#    from jobuser_df
#    right join Website_visits_df
#    on
#        jobuser_df.job_agent_id = Website_visits_df.job_agent_id
#"""
#)

#Website_visits_jobuser_df.drop("user_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
#3)	If there is only a user_hash_id then match it to a user_id only if that user_hash_id is used with only one user_id

# take all user_ids that are only related to 1 user_id:
# select user_hash_id, count(distinct user_id) from table
# filter, where count(distinct user_id) = 1
# take the resulting list of user_has_ids to join
user_hash_df = spark.sql("""
    with hash_count as (
        select user_hash, count(distinct user_id) as n_UID
        from presentation.adobe_user_id_hash_de
        group by user_hash)
    select A.user_hash, A.user_id as user_id_hash
    from presentation.adobe_user_id_hash_de A
    right join hash_count B 
    on 
        A.user_hash = B.user_hash 
        and B.n_UID = 1""")

#user_hash_df.show(n=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
#Website_visits_df_final: it includes user_id from Website_visits_df, userlogin_id from jobuser_df and user_id_hash from user_hash_df
# this can be simplified as:
Website_visits_df_final = Website_visits_jobuser_df.join(user_hash_df, "user_hash", "left")

# user_hash_df.registerTempTable("user_hash_df")
#Website_visits_jobuser_df.registerTempTable("Website_visits_jobuser_df")

#Website_visits_df_final = spark.sql("""
#    select *
#    from Website_visits_jobuser_df
#    left join user_hash_df
#    on
#        user_hash_df.user_hash = Website_visits_jobuser_df.user_hash
#""")

#Website_visits_df_final.show(n=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
#from Website_visits_df_final map user_ids from user_hash_df and jobuser_df into Website_visits_df to fill null values
#Note: mapping didn't work using several when/otherwise inside each other, so the code is written in seperated parts!

# in pyspark notation without using spark.sql:
Website_visits_df_final1 = Website_visits_df_final.select("user_id", "userlogin_id", "user_id_hash", "visit_date")

#Website_visits_df_final.registerTempTable("Website_visits_df_final")
#Website_visits_df_final1 = spark.sql("""
#    select 
#        user_id,
#        userlogin_id,
#        user_id_hash,
#        visit_date
#    from Website_visits_df_final
#""")
#Website_visits_df_final1.registerTempTable("Website_visits_df_final1")

Website_visits_df_final2 = Website_visits_df_final1.withColumn('new_user_id', 
                                                               F.when(F.col('user_id').isNotNull(), col('user_id')). \
                                                               otherwise(F.when(F.col('userlogin_id').isNotNull(), col('userlogin_id'))))

Website_visits_df_final3 = Website_visits_df_final2.withColumn('new_user_id_1', 
                                                               F.when(F.col('new_user_id').isNotNull(), col('new_user_id')). \
                                                               otherwise(F.when(F.col('user_id_hash').isNotNull(), col('user_id_hash'))))

Final_Website_visits_df = Website_visits_df_final3.filter(F.col("new_user_id_1").isNotNull()).select(F.col("new_user_id_1").alias("user_id"), "visit_date")

#Website_visits_df_final3.registerTempTable("Website_visits_df_final3")
#Filter null user ids from website visit data to speed up further joins
#Final_Website_visits_df = spark.sql("""
#    select 
#        new_user_id_1 as user_id,
#        visit_date
#    from Website_visits_df_final3
#    where new_user_id_1 IS NOT NULL
#""")
                                

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
#Final_Website_visits_df.show(n=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+
|user_id|visit_date|
+-------+----------+
|2557149|2016-07-21|
|6032768|2017-10-11|
|6032768|2017-10-04|
|6032768|2017-10-04|
|6032768|2017-10-04|
+-------+----------+
only showing top 5 rows

In [24]:
#Bring website visit (without null user_id) and email data frames together
#web_email_df: Join Final_Website_visits_df table with email_log_df

# again in pyspark notation without temp tables and sql
web_email_df1 = email_log_df.join(Final_Website_visits_df, "user_id", "left")

#Final_Website_visits_df.registerTempTable("Final_Website_visits_df")
#email_log_df.registerTempTable("email_log_df")
#web_email_df1 = spark.sql("""
#    select *
#    from email_log_df as df1
#    left join Final_Website_visits_df as df2
#    on df1.user_id = df2.user_id
#""")

# select columns from the same table (i am not sure what your original select does)
web_email_df1 = web_email_df1.select("user_id", "reg_date", "validated_account", "considered_date","sub_state","FourtyFiveDaysAgo", "visit_date")
#web_email_df1 = web_email_df1.select(email_log_df.user_id,
#                                    email_log_df.reg_date,
#                                    email_log_df.validated_account,
#                                    email_log_df.considered_date,
#                                    email_log_df.sub_state, 
#                                    email_log_df.FourtyFiveDaysAgo,
#                                    Final_Website_visits_df.visit_date)

# this shouldnt be replaced - even though there are probably no nulls in user_id: web_email_df1 = web_email_df1.fillna({'user_id':'0'})
# this shouldnt be replaced - even though there are probably no nulls in reg_date: web_email_df1 = web_email_df1.fillna({'reg_date':'0'})
# this shouldnt be replaced - even though there are probably no nulls in considered_date:web_email_df1 = web_email_df1.fillna({'considered_date':'0'})
web_email_df1 = web_email_df1.fillna({'validated_account':'0'})
web_email_df1 = web_email_df1.fillna({'sub_state':'0'})
web_email_df1 = web_email_df1 = web_email_df1.fillna({'FourtyFiveDaysAgo':'0'})
# this MUST NOT be replaced: web_email_df1 = web_email_df1.fillna({'visit_date':'0'})

#web_email_df.show(n=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# Updated: second UK rule, which also looks at account validation + existing subscription
#ADD UK Rule considered:

#Adding FourtyFiveVisits collumn to calculate UK_rule that is the number of website visits up to 45 days ago, 
#i.e. between 20/02/2020 and 5/04/2020 > did the user visit the website within the last 45 days

#add date_difference which is 45 days before user visiting the website
web_email_df1 = web_email_df1.select("user_id", "reg_date", "validated_account", "considered_date", "sub_state", "FourtyFiveDaysAgo", "visit_date"). \
    distinct(). \
    withColumn('date_difference', datediff('FourtyFiveDaysAgo', 'visit_date'))

#add UK_rull_considered based on date_difference
web_email_df_UK1 = web_email_df1.select("user_id", "reg_date", "validated_account", "considered_date", "sub_state", "FourtyFiveDaysAgo", "visit_date", "date_difference"). \
    distinct(). \
    groupBy("user_id", "reg_date", "validated_account", "considered_date", "sub_state"). \
    agg(F.sum(F.when(F.col("date_difference") <= 45, F.lit(1)).otherwise(0)).alias("FourtyFiveVisits")). \
    withColumn('UK_rule_hasVisit', F.when(F.col('FourtyFiveVisits') >=  1, 1).otherwise(0)).\
    withColumn('UK_rule_hasVisitSubValidated_account', F.when((F.col('FourtyFiveVisits') >=  1) & (F.col("validated_account") > 0) & (F.col("sub_state") > 0), 1).otherwise(0))

web_email_df_UK = web_email_df_UK1.select("user_id", "reg_date", "validated_account", "considered_date", "sub_state","UK_rule_hasVisit", "UK_rule_hasVisitSubValidated_account")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
#summary table: How many email audience is considered within UK rule per consider day
Grouped_web_email_df_UK= web_email_df_UK.groupBy("considered_date").\
    agg(F.countDistinct("user_id").alias("registrations"),
        F.countDistinct(F.when(F.col("validated_account") >=1, F.col("user_id"))).alias("validated_accounts"),
        F.countDistinct(F.when(F.col("sub_state") >=1, F.col("user_id"))).alias("subscriptions"),
        F.countDistinct(F.when(F.col("UK_rule_hasVisit") >=1, F.col("user_id"))).alias("UK_rule_visits_only"),
        F.countDistinct(F.when(F.col("UK_rule_hasVisitSubValidated_account") >=1, F.col("user_id"))).alias("UK_rule_visitsSubVal")).\
    sort("considered_date").show(100)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------------+------------------+-------------+-------------------+--------------------+
|considered_date|registrations|validated_accounts|subscriptions|UK_rule_visits_only|UK_rule_visitsSubVal|
+---------------+-------------+------------------+-------------+-------------------+--------------------+
|     2020-01-01|         1258|               736|          517|               1146|                 416|
|     2020-01-02|         4058|              2478|         1825|               3578|                1433|
|     2020-01-03|         6808|              4348|         3248|               6072|                2562|
|     2020-01-04|         9025|              5922|         4392|               8079|                3474|
|     2020-01-05|        11626|              7666|         5648|              10443|                4467|
|     2020-01-06|        15114|              9979|         7355|              13619|                5828|
|     2020-01-07|        18609|             12

In [ ]:
# next steps: 
# 1) create output in excel (above table)
# 2) compare outcome against state in subscription table (use e.g. hue to understand subscription table)
# 3) user ssa data as alternative to website visits

In [26]:
# next steps: 
# 1) create output in excel (above table)
Grouped_web_email_df_UK= web_email_df_UK.groupBy("considered_date").\
    agg(F.countDistinct("user_id").alias("registrations"),
        F.countDistinct(F.when(F.col("validated_account") >=1, F.col("user_id"))).alias("validated_accounts"),
        F.countDistinct(F.when(F.col("sub_state") >=1, F.col("user_id"))).alias("subscriptions"),
        F.countDistinct(F.when(F.col("UK_rule_hasVisit") >=1, F.col("user_id"))).alias("UK_rule_visits_only"),
        F.countDistinct(F.when(F.col("UK_rule_hasVisitSubValidated_account") >=1, F.col("user_id"))).alias("UK_rule_visitsSubVal")).\
    sort("considered_date")

# Grouped_web_email_df_UK.repartition(1).write.format('com.databricks.spark.csv'). \
#                 option("header", "true").mode('overwrite'). \
#                 save('s3://live-emr-notebooks-stst/Parastoo/e-70RUMRMNWLHH4YBMCBQ2PUXAB/web_email_df_UK.csv')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# 2) compare outcome against state in subscription table (use e.g. hue to understand subscription table)

#2.1) Compare subscription table (left side) data and web_email_df_UK table (right side) on: 
#### 2.1.1) How many records found in subscriptiondate_df and the matched records from web_email_df_UK (left join): 286145
#### 2.1.2) How many records found in web_email_df_UK and matched records from subscriptiondate_df when (right join): 383870*
#### 2.1.3) How many common records found in both subscriptiondate_df and web_email_df_UK (inner join): 215613*

#finding(s):
    #1. There are 168257 (383870 - 215613) distinct users that are covered by our UK rule audience but are NOT in subscription table
        #This counts for losing about 10k (i.e., 9897) distinct user-ids every week, if our udience rule is NOT implemented.

    #2. There are 70532 (286145 - 215613) distinct users in subscription table that our UK_rule does NOT cover them. 
        # , reletively small number (70k in 17 weeks, about 4k per week), 
        # but the question is why? Should/How to optimise the audience rule to cover these users?

#--------------------------------------------------------------------------------------------------------------------
#2.2) sub_web_email_df_UK_lastDay: inner join subscription table and web_email_df_UK and filter for 2020-05-08:

    # 2.2.1) number of distinct user_id in sub_web_email_df_UK_right on one day (2020-05-08)
        # with registration: 383310* 
        # with registration + validated account (UK_rule_hasVisit): 277956
        # with registration + validated account + subscription (UK_rule_visitsSubVal): 136702
        
    # 2.2.2) number of distinct user_id in sub_web_email_df_UK_inner on one day (2020-05-08)
        # with registration: 215143*
        # with registration + validated account (UK_rule_hasVisit): 171664
        # with registration + validated account + subscription (UK_rule_hasVisitSubValidated): 136702

#finding(s):
    # UK_rule_hasVisitSubValidated generates similar number of distinct users (136702) in both inner and right joins on 2020-05-08
    
#Note: 
    #*550 users are not matched as registered. 
#--------------------------------------------------------------------------------------------------------------------        
#2.3) Contingency table UK rule 1 vs. active (in subscription). and uk rule 2 vs. active (in subscription).
# Note: I'm not sure about this logic. Pl check!
 
    #2.3.1) Contingency table UK rule 1 vs. active (in subscription).
        #nr of 1: 171664
        #nr of 0: 43479
    
    #2.3.2) Contingency table uk rule 2 vs. active (in subscription).
        #nr of 1: 136702
        #nr of 0: 78441      

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
#2.1) subs_web_email_df_UK_finalDay: import data from subscription table and left join with web_email_df_UK_finalDay (last day in web_email_df_UK)
subscriptiondate_df = spark.sql("""
    select 
        userlogin_id as user_id
        , isactive 
        , to_date(subscriptiondate) as sub_date
        , dailysentcounter
        , frequency
        , manualunsubscribe   
    from stepstonede.subscription
    where subscriptiondate >= '2020-01-01' 
    """)

#subscriptiondate_df.show(n=10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
#2.1.1) How many records found in the left side (subscriptiondate_df) and the common records found in web_email_df_UK (left join)

#leftjoin subscriptiondate_df with web_email_df_UK:
#to get all the records found in the left side (subscriptiondate_df) and the common records found in (web_email_df_UK) 
sub_web_email_df_UK_left = subscriptiondate_df.join(web_email_df_UK, "user_id", "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
#how many distinct user_id found in subscriptiondate_df and the common records found in web_email_df_UK?
sub_web_email_df_UK_left.agg(F.countDistinct("user_id").alias("subscribed_constructed")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+
|subscribed_constructed|
+----------------------+
|                286145|
+----------------------+

In [28]:
#2.1.2) How many records found in web_email_df_UK and return data from subscriptiondate_df when it matches (right join)

#to select all records from web_email_df_UK and return data from subscriptiondate_df when it matches
sub_web_email_df_UK_right = subscriptiondate_df.join(web_email_df_UK, "user_id", "right")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# how many distinct user_id found in web_email_df_UK and found in subscriptiondate_df
sub_web_email_df_UK_right.agg(F.countDistinct("user_id").alias("constructed_rule_only")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+
|constructed_rules_only|
+----------------------+
|                383870|
+----------------------+

In [29]:
# 2.1.3) How many common records found in both subscriptiondate_df and web_email_df_UK (inner join)

# the common records found in both subscriptiondate_df and web_email_df_UK
sub_web_email_df_UK_inner = subscriptiondate_df.join(web_email_df_UK, "user_id", "inner")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# how many distinct user_id found in both web_email_df_UK and subscriptiondate_df
sub_web_email_df_UK_inner.agg(F.countDistinct("user_id").alias("common_users_subscription_constructed")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------------+
|similar_subscription_constructed|
+--------------------------------+
|                          215613|
+--------------------------------+

In [ ]:
#2.1) Compare subscription table data and our UK_rule audience logic in this script:

#Findings from the data that is retrieved between (2020-01-01, 2020-05-08) a period of 4 months and 1 week:
# 2.1.1) There are 215613 common distinct user_id in subscription table and UK_rule audience (inner join)
# 2.1.2) There are 383870 records that include all of UK_rule audience and the matched users in subscription table (right join)
# 2.1.3) There are 286145 records that include all of subscription table and the matched users in UK_rule audience (left join)

# 2 Conclusions: 
#1. There are 168257 (383870 - 215613) distinct users that are covered by our UK rule audience but are NOT in subscription table
#   This counts for losing about 10k (i.e., 9897) distinct user-ids on average every week, if the udience rule is NOT implemented.

#2. There are 70532 (286145 - 215613) distinct users in subscription table that our UK_rule does NOT cover them. 
    # Reletively small number (70k in 17 weeks, about 4k per week) but the question is why? 
    # How to optimise the audience rule to cover these users?

In [30]:
# 2.3. Compare subscription table data and our UK_rule audience logic in one day ('2020-05-08')

# 2.3.1.) 2020-05-08: all of the users in web_email_df_UK and the matched users from subscription table 
#filter sub_web_email_df_UK_right to show only last subscirbed date (i.e., '2020-05-08')
sub_web_email_df_UK_right_lastDay = sub_web_email_df_UK_right.filter(F.col("considered_date") == '2020-05-08')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
# number of distinct user_id in sub_web_email_df_UK_right on one day (2020-05-08)
    # with registration: 383310
    # with registration + validated account (UK_rule_hasVisit): 277956
    # with registration + validated account + subscription (UK_rule_visitsSubVal): 136702

sub_web_email_df_UK_lastDay_right= sub_web_email_df_UK_right_lastDay.groupBy("considered_date").\
    agg(F.countDistinct("user_id").alias("registrations"),
        F.countDistinct(F.when(F.col("validated_account") >=1, F.col("user_id"))).alias("validated_accounts"),
        F.countDistinct(F.when(F.col("sub_state") >=1, F.col("user_id"))).alias("subscriptions"),
        F.countDistinct(F.when(F.col("UK_rule_hasVisit") >=1, F.col("user_id"))).alias("UK_rule_visits_only"),
        F.countDistinct(F.when(F.col("UK_rule_hasVisitSubValidated_account") >=1, F.col("user_id"))).alias("UK_rule_visitsSubVal")).\
    sort("considered_date").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------------+------------------+-------------+-------------------+--------------------+
|considered_date|registrations|validated_accounts|subscriptions|UK_rule_visits_only|UK_rule_visitsSubVal|
+---------------+-------------+------------------+-------------+-------------------+--------------------+
|     2020-05-08|       383310|            218032|       214974|             277956|              136702|
+---------------+-------------+------------------+-------------+-------------------+--------------------+

In [37]:
# 2.3.2) 2020-05-08: only matched users in web_email_df_UK and subscription table 
#filter sub_web_email_df_UK_right to show only last subscirbed date (i.e., '2020-05-08')
sub_web_email_df_UK_inner_lastDay = sub_web_email_df_UK_inner.filter(F.col("considered_date") == '2020-05-08')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
#2.3) Contingency table UK rule 1 vs. active (in subscription). and uk rule 2 vs. active

sub_web_email_df_UK_lastDay_inner = sub_web_email_df_UK_inner_lastDay.groupBy("user_id").\
    agg(F.countDistinct("user_id").alias("registrations"),
        F.countDistinct(F.when(F.col("validated_account") >=1, F.col("user_id"))).alias("validated_accounts"),
        F.countDistinct(F.when(F.col("sub_state") >=1, F.col("user_id"))).alias("subscriptions"),
        F.countDistinct(F.when(F.col("UK_rule_hasVisit") >=1, F.col("user_id"))).alias("UK_rule_visits_only"),
        F.countDistinct(F.when(F.col("UK_rule_hasVisitSubValidated_account") >=1, F.col("user_id"))).alias("UK_rule_visitsSubVal")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------------+------------------+-------------+-------------------+--------------------+
|considered_date|registrations|validated_accounts|subscriptions|UK_rule_visits_only|UK_rule_visitsSubVal|
+---------------+-------------+------------------+-------------+-------------------+--------------------+
|     2020-05-08|       215143|            175198|       214974|             171664|              136702|
+---------------+-------------+------------------+-------------+-------------------+--------------------+

In [46]:
#2.3) Contingency table UK rule 1 vs. active (in subscription). and uk rule 2 vs. active

#2.3.1) Contingency table UK rule 1 vs. active (in subscription).
    #nr of 1: 171664
    #nr of 0: 43479
sub_web_email_df_UK_lastDay_inner_cont_1st_rule = sub_web_email_df_UK_inner_lastDay.stat.crosstab("UK_rule_hasVisit", "UK_rule_hasVisit").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------+-----+------+
|UK_rule_hasVisit_UK_rule_hasVisit|    0|     1|
+---------------------------------+-----+------+
|                                1|    0|171664|
|                                0|43479|     0|
+---------------------------------+-----+------+

In [47]:
#2.3.2) Contingency table uk rule 2 vs. active (in subscription).
    #nr of 1: 136702
    #nr of 0: 78441
sub_web_email_df_UK_lastDay_inner_cont_1st_rule = sub_web_email_df_UK_inner_lastDay.stat.crosstab("UK_rule_hasVisitSubValidated_account", "UK_rule_hasVisitSubValidated_account").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------------------------------+-----+------+
|UK_rule_hasVisitSubValidated_account_UK_rule_hasVisitSubValidated_account|    0|     1|
+-------------------------------------------------------------------------+-----+------+
|                                                                        1|    0|136702|
|                                                                        0|78441|     0|
+-------------------------------------------------------------------------+-----+------+

In [ ]:
#extra analysis:
# what nr of considered users at some point in time ( say, last friday) , have had no activity in the past:
    #1-3 days
    #3-14 days
    #14-42 days
    #42-84 days
    #84-191 days
    #> 191 days
    

In [ ]:
# 3) user ssa data as alternative to website visits